**ReadME**  
**Overview of the Entire JupyterNotebook**  
Section 1: Required Libraries  
Section 2: Data Processing
* 2.1 Single Asset Data Processing  
* 2.2 Multi Asset Data Processing  

Section 3: LSTM Machine Learning Estimator  
* 3.1 Pre-Processing
* 3.2 LSTM Model SetUp

Section 4: Cluster Asset Main Case

**Section 1: Required Libraries**

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Dense
from tensorflow.keras.regularizers import l2

2023-12-04 10:20:17.881781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Section 2: Data Processing**  
Section 2.1 Single Asset Data Processing

In [ ]:
def get_historical_returns(ticker, start_date, end_date, frequency="monthly"):
    'Function to fetch Historical Price data and compute returns'

    data = yf.download(ticker,start=start_date, end=end_date)

    # Calculate Daily Returns
    daily_data = data.copy()
    daily_data['Return'] = daily_data['Close'].pct_change()
    daily_returns = daily_data[['Return']].dropna()

    # Calculate Monthly Returns
    monthly_data = data.copy()
    monthly_data['Return'] = monthly_data['Close']
    monthly_data = monthly_data['Return'].resample('M').last()
    monthly_returns = monthly_data.pct_change()
    monthly_returns = monthly_returns.dropna()

    if frequency == "daily": return daily_returns
    if frequency == "monthly": return monthly_returns

    return monthly_data

def resample_quaterly_data(quaterly_data, target_data):
    'Repeat the quaterly available ratios to same frequency as target return'

    quaterly_data.index = pd.to_datetime(quaterly_data.index)
    target_data.index = pd.to_datetime(target_data.index)

    # Resample the quaterly data to daily frequency using Forward Fill
    quaterly_data.index = quaterly_data.index + pd.DateOffset(days=1)
    aligned_quaterly_data = quaterly_data.reindex(target_data.index, method='ffill')

    aligned_quaterly_data = aligned_quaterly_data.dropna()
    return aligned_quaterly_data


def load_features(path_to_file, ticker, start_date, end_date):
    'Function to Load all features for a single company'

    # Load the Excel file and read Data from the file
    file_path = path_to_file + ticker + '.xlsx'
    sheet_name = ticker + '-US'
    data = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')

    # Remove rows with any NaN values
    # Because time frame is longer, cannot apply this
    # data = data.dropna()

    # Reset the index of the DataFrame and drop the old index
    data = data.reset_index(drop=True)

    data = data.set_index('Date').T
    data.index = pd.to_datetime(data.index, format='%b \'%y')
    data.index = data.index + pd.offsets.MonthEnd()
    ratio_data = data.apply(pd.to_numeric)

    # Select a few columns
    pe_column = 'Price/Earnings'
    pb_column = 'Price/Book Value'
    roa_column = 'Return on Assets'
    roe_column = 'Return on Equity '
    fcf_column = 'Free Cash Flow per Share'
    ratio_data = data[[pe_column, pb_column, roa_column, roe_column, fcf_column]]

    # Drop N/A dates
    # Removing rows with any NaN values
    ratio_data = ratio_data.dropna()

    # Process Return Data
    returns_data = get_historical_returns(ticker, start_date, end_date)
    adjusted_ratio_data = resample_quaterly_data(ratio_data, returns_data)
    features = pd.concat([adjusted_ratio_data, returns_data],axis=1)

    return features

Section 2.2 Multi Asset Data Processing

In [ ]:
def multi_df(path_to_file, ticker_list, start_date, end_date):
    company_data = {}
    for ticker in ticker_list:
        company_data[ticker] = load_features(path_to_file, ticker, start_date, end_date)

    # Initialize a list to hold DataFrames with the new multi-index
    multi_index_dfs = []

    for company, df in company_data.items():
        # Set the company name as an additional level in the index
        df_multi_index = df.copy()
        df_multi_index['Company'] = company
        df_multi_index.set_index(['Company', df_multi_index.index], inplace=True)

        # Append to the list
        multi_index_dfs.append(df_multi_index)

    # Concatenate all DataFrames into a single multi-index DataFrame
    final_df = pd.concat(multi_index_dfs)

    return final_df

**Section 3: LSTM Machine Learning Estimator**  
Section 3.1 Pre-Processing

In [ ]:
def create_sequences(features, targets, seq_length):
    'Function to create sequence'
    'Need to define the sequence length: e.g. using 4 quaters to predict the next quater'

    xs = []
    ys = []

    for i in range(len(features)-seq_length):
        x = features[i:(i+seq_length)]
        y = targets.iloc[i+seq_length]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

Section 3.2 LSTM Model SetUp

In [ ]:
# LSTM Model Set Up

# Model architecture
model = tf.keras.Sequential([
    LSTM(512, return_sequences=True),
    Dropout(0.02),
    LSTM(256, return_sequences=True),
    LSTM(128),
    Dense(1, activation='linear', kernel_regularizer=l2(0.0005))
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='mean_squared_error')

**Section 4: Cluster 15 Stock Test Case**


Step 1: Define Input and Parameters  
**Please change the path_to_file to the appropriate path in the running environment**

In [ ]:
# 1. File Path
path_to_file = 'Clustering_Tickers/'
# 2. Ticker List
Cluster_stocks = ['AMC', 'AME','CRL', 'DVN', 'DFS', 'DTE', 'FERG', 'GIS', 'HD', 'MSFT', 'PRU', 'RUN', 'TMO', 'RARE', 'VIRT']
# 3. Target Time Frame
start_date = '2016-09-30'
end_date = '2021-09-30'
# 4. Sequence Length
seq_length = 6
# 5. Training and Validation Set Split Ratio
train_ratio = 0.8
# 6. Num Epoch and Num Batch
num_epoch = 20
num_batch = 30

Step2: Pre-Processing

In [ ]:
# Loading Phase: Took a while to run this (Don't Rerun)
final_df = multi_df(path_to_file, Cluster_stocks, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
multi_df_final = final_df
multi_return_final = pd.DataFrame(multi_df_final['Return'])

In [ ]:
features = multi_df_final
targets = multi_return_final
multi_X, multi_y = create_sequences(features, targets, seq_length)

Step3: Train Model on Training and Validation Sets

In [ ]:
# Variable in Use and Constant Define
X = multi_X
y = multi_y

# Split data into training and validation sets
train_size = int(len(X) * train_ratio)
X_train, X_vali = X[:train_size], X[train_size:]
y_train, y_vali = y[:train_size], y[train_size:]

# Train the model
model.fit(X_train, y_train, epochs=num_epoch, batch_size=num_batch, validation_data=(X_vali, y_vali))

Epoch 1/20
24/24 [==============================] - 12s 168ms/step - loss: 1.2604 - val_loss: 0.0179
Epoch 2/20
24/24 [==============================] - 2s 94ms/step - loss: 0.0629 - val_loss: 0.0296
Epoch 3/20
24/24 [==============================] - 2s 104ms/step - loss: 0.0582 - val_loss: 0.0183
Epoch 4/20
24/24 [==============================] - 3s 118ms/step - loss: 0.0575 - val_loss: 0.0185
Epoch 5/20
24/24 [==============================] - 2s 102ms/step - loss: 0.0594 - val_loss: 0.0294
Epoch 6/20
24/24 [==============================] - 2s 96ms/step - loss: 0.0569 - val_loss: 0.0246
Epoch 7/20
24/24 [==============================] - 2s 100ms/step - loss: 0.0551 - val_loss: 0.0196
Epoch 8/20
24/24 [==============================] - 3s 112ms/step - loss: 0.0560 - val_loss: 0.0201
Epoch 9/20
24/24 [==============================] - 2s 99ms/step - loss: 0.0553 - val_loss: 0.0223
Epoch 10/20
24/24 [==============================] - 3s 111ms/step - loss: 0.0547 - val_loss: 0.0143
E

In [ ]:
# Check Trianing Set Output
# X_pred, y_pred = X_train, y_train
# predictions = model.predict(X_pred)
# print(predictions)
# print(X_train)

Step4: Evaluate the Model on Test Set

In [ ]:
# Test Period 1: September 30 2021 to July 30 2023
# Test Period 2: March 14, 2023 to July 31 2023
# Test Period 3: May 01 2023 to July 31 2023

start_t1 = '2021-09-30'
end_t1 = '2023-07-31'

start_t2 = '2023-03-14'
end_t2 = '2023-07-31'

start_t3 = '2023-05-01'
end_t3 = '2023-07-31'

In [ ]:
# Loading Test Phase: Took a while to run this (Don't Rerun)
final_df_t1 = multi_df(path_to_file, Cluster_stocks, start_t1, end_t1)
# final_df_t2 = multi_df(path_to_file, ChatGPT_stocks, start_t2, end_t2)
# final_df_t3 = multi_df(path_to_file, ChatGPT_stocks, start_t3, end_t3)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
multi_df_t1 = final_df_t1
multi_return_t1 = pd.DataFrame(multi_df_t1['Return'])
seq_length = 6
multi_X_t1, multi_y_t1 = create_sequences(multi_df_t1, multi_return_t1, seq_length)

In [ ]:
def generate_sequence_mapping(df,freq):
    sequences = []
    sequence_mappings = []

    for company in df.index.get_level_values(0).unique():
        # Get the data for the current company
        company_data = df.xs(company, level='Company')

        # Create 6-month sequences and record their mappings
        for i in range(len(company_data) - (freq-1)):
            sequence = company_data.iloc[i:i+freq]
            if sequence.shape[0] == freq:  # Ensure each sequence has 6 months
                sequences.append(sequence.drop(columns='Return').values)  # Add the sequence to the list, excluding 'Return' if it's not an input feature
                end_date = sequence.index[-1]  # The end date of the sequence
                sequence_mappings.append((company, end_date))  # Record the mapping
    return sequence_mappings


**Evaluation for Each test Period Execution parts**

In [ ]:
df = multi_df_t1
X_test, y_test = multi_X_t1, multi_y_t1

# Parse accordingly before feeding into the model
sequence_mappings = generate_sequence_mapping(df,6)
print(sequence_mappings)

[('AMC', Timestamp('2022-03-31 00:00:00')), ('AMC', Timestamp('2022-04-30 00:00:00')), ('AMC', Timestamp('2022-05-31 00:00:00')), ('AMC', Timestamp('2022-06-30 00:00:00')), ('AMC', Timestamp('2022-07-31 00:00:00')), ('AMC', Timestamp('2022-08-31 00:00:00')), ('AMC', Timestamp('2022-09-30 00:00:00')), ('AMC', Timestamp('2022-10-31 00:00:00')), ('AMC', Timestamp('2022-11-30 00:00:00')), ('AMC', Timestamp('2022-12-31 00:00:00')), ('AMC', Timestamp('2023-01-31 00:00:00')), ('AMC', Timestamp('2023-02-28 00:00:00')), ('AMC', Timestamp('2023-03-31 00:00:00')), ('AMC', Timestamp('2023-04-30 00:00:00')), ('AMC', Timestamp('2023-05-31 00:00:00')), ('AMC', Timestamp('2023-06-30 00:00:00')), ('AMC', Timestamp('2023-07-31 00:00:00')), ('AME', Timestamp('2022-03-31 00:00:00')), ('AME', Timestamp('2022-04-30 00:00:00')), ('AME', Timestamp('2022-05-31 00:00:00')), ('AME', Timestamp('2022-06-30 00:00:00')), ('AME', Timestamp('2022-07-31 00:00:00')), ('AME', Timestamp('2022-08-31 00:00:00')), ('AME', Ti

In [ ]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Mean Squared Error: {loss}")

11/11 [==============================] - 0s 23ms/step - loss: 0.0182
Mean Squared Error: 0.018247663974761963


In [ ]:
# Make predictions
predictions = model.predict(X_test)

11/11 [==============================] - 0s 23ms/step


In [ ]:
# Prediction Result Mapping back to Each Company
results_list = []

# Use the shorter length of the two lists to avoid IndexError
min_length = min(len(predictions), len(sequence_mappings))

for i in range(min_length):
    company, end_date = sequence_mappings[i]
    expected_return = predictions[i][0]
    results_list.append({'Company': company, 'EndDate': end_date, 'ExpectedReturn': expected_return})

results_df = pd.DataFrame(results_list)

In [ ]:
# Generate Covariance Matrix

# Pivot the DataFrame so that each company's returns form a column
pivoted_df = results_df.pivot(index='EndDate', columns='Company', values='ExpectedReturn')
pivoted_df.isna().any()

# Calculate the covariance matrix
# covariance_matrix now contains the covariance of returns between companies
covariance_matrix = pivoted_df.cov()
covariance_matrix.to_csv('Saved_mu_Q/LSTM_Cluster_Q_1.csv', index=False, header=False)

In [ ]:
# Ouput the final Average Expected return for each company
# Group by 'Company' and calculate the mean of 'ExpectedReturn'
# average_returns now contains the average expected return for each company
average_returns = results_df.groupby('Company')['ExpectedReturn'].mean()
average_returns.to_csv('Saved_mu_Q/LSTM_Cluster_mu_1.csv', index=False, header=False)

**For Test Period 2: March 14, 2023 to July 31 2023**


In [ ]:
results_df['EndDate'] = pd.to_datetime(results_df['EndDate'])
filtered_df_t2 = results_df[results_df['EndDate'] >= '2023-03-14']
# filtered_df_t2.isna().any()

In [ ]:
# Generate Covariance Matrix
# Pivot the DataFrame so that each company's returns form a column
pivoted_df = filtered_df_t2.pivot(index='EndDate', columns='Company', values='ExpectedReturn')

# Calculate the covariance matrix
# covariance_matrix now contains the covariance of returns between companies
covariance_matrix = pivoted_df.cov()
covariance_matrix.to_csv('Saved_mu_Q/LSTM_Cluster_Q_2.csv', index=False, header=False)

In [ ]:
# Ouput the final Average Expected return for each company
# Group by 'Company' and calculate the mean of 'ExpectedReturn'
# average_returns now contains the average expected return for each company
average_returns = filtered_df_t2.groupby('Company')['ExpectedReturn'].mean()
average_returns.to_csv('Saved_mu_Q/LSTM_Cluster_mu_2.csv', index=False, header=False)

**For Test Period 3: Test Period 3: May 01 2023 to July 31 2023**


In [ ]:
results_df['EndDate'] = pd.to_datetime(results_df['EndDate'])
filtered_df_t3 = results_df[results_df['EndDate'] >= '2023-05-01']

In [ ]:
# Generate Covariance Matrix

# Pivot the DataFrame so that each company's returns form a column
pivoted_df = filtered_df_t3.pivot(index='EndDate', columns='Company', values='ExpectedReturn')

# Calculate the covariance matrix
# covariance_matrix now contains the covariance of returns between companies
covariance_matrix = pivoted_df.cov()
# print(covariance_matrix)
covariance_matrix.to_csv('Saved_mu_Q/LSTM_Cluster_Q_3.csv', index=False, header=False)

In [ ]:
# Ouput the final Average Expected return for each company
# Group by 'Company' and calculate the mean of 'ExpectedReturn'
# average_returns now contains the average expected return for each company
average_returns = filtered_df_t3.groupby('Company')['ExpectedReturn'].mean()
# print(average_returns)
average_returns.to_csv('Saved_mu_Q/LSTM_Cluster_mu_3.csv', index=False, header=False)